In [2]:
from IPython.display import HTML
import numpy as np
import urllib3
from bs4 import BeautifulSoup as soup 
import pandas as pd

import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns
sns.set_context("talk")
sns.set_style("white")

# Part I - Extraction of data from MTA's website
## Let's scraping the MTA website to get all the URLs that have weekly turnstile data in txt format

In [159]:
url = 'http://web.mta.info/developers/turnstile.html'
manager = urllib3.PoolManager()
response = manager.request('GET', url)

In [160]:
#store the html page code
html = soup(response.data, 'html.parser')

In [161]:
#let's check if we can access HTML tags ad the data
html.head.title

<title>mta.info | Turnstile Data</title>

In [162]:
#the URLs are in a div with class=span-84 last
#storing the div
data_files = html.find_all('div', {'class':'span-84 last'})

In [166]:
#get all anchor tags which have the URLs
data_files[0].find_all('a')[:10]

[<a href="data/nyct/turnstile/turnstile_180310.txt">Saturday, March 10, 2018</a>,
 <a href="data/nyct/turnstile/turnstile_180303.txt">Saturday, March 03, 2018</a>,
 <a href="data/nyct/turnstile/turnstile_180224.txt">Saturday, February 24, 2018</a>,
 <a href="data/nyct/turnstile/turnstile_180217.txt">Saturday, February 17, 2018</a>,
 <a href="data/nyct/turnstile/turnstile_180210.txt">Saturday, February 10, 2018</a>,
 <a href="data/nyct/turnstile/turnstile_180203.txt">Saturday, February 03, 2018</a>,
 <a href="data/nyct/turnstile/turnstile_180127.txt">Saturday, January 27, 2018</a>,
 <a href="data/nyct/turnstile/turnstile_180120.txt">Saturday, January 20, 2018</a>,
 <a href="data/nyct/turnstile/turnstile_180113.txt">Saturday, January 13, 2018</a>,
 <a href="data/nyct/turnstile/turnstile_180106.txt">Saturday, January 06, 2018</a>]

#### Store all the URLs in list

In [168]:
list_of_urls = []
for elem in data_files[0].find_all('a'):
    list_of_urls.append(elem.get('href'))

In [169]:
list_of_urls[:5]

['data/nyct/turnstile/turnstile_180310.txt',
 'data/nyct/turnstile/turnstile_180303.txt',
 'data/nyct/turnstile/turnstile_180224.txt',
 'data/nyct/turnstile/turnstile_180217.txt',
 'data/nyct/turnstile/turnstile_180210.txt']

## Time to create a dataframe using above URLs
### Our analysis is only going to focus on year 2017. So we need to get the URLs for year 2017 and create a DataFrame

In [15]:
print(list_of_urls[0], "\nlength is", len(list_of_urls[0]))

data/nyct/turnstile/turnstile_180310.txt 
length is 40


In [20]:
list_of_urls[0][30:32]

'18'

#### So we need to grab the characters at index 30 & 31 to get the year

In [23]:
year_2017 = list()

In [26]:
for item in list_of_urls:
    if item[30:32] == '17':
        year_2017.append(item)


In [28]:
len(year_2017)

52

In [29]:
base_url = "http://web.mta.info/developers/"
base_url+year_2017[0]

'http://web.mta.info/developers/data/nyct/turnstile/turnstile_171230.txt'

In [32]:
mta = pd.DataFrame()
countdown = len(year_2017) + 1

for i in year_2017:
    temp = pd.read_csv(base_url+i)
    mta = mta.append(temp, ignore_index=True) #appending will result in index getting repeated and hence they need to be ignored
    countdown = countdown - 1
    print("Done with week",countdown)

Done with week 52
Done with week 51
Done with week 50
Done with week 49
Done with week 48
Done with week 47
Done with week 46
Done with week 45
Done with week 44
Done with week 43
Done with week 42
Done with week 41
Done with week 40
Done with week 39
Done with week 38
Done with week 37
Done with week 36
Done with week 35
Done with week 34
Done with week 33
Done with week 32
Done with week 31
Done with week 30
Done with week 29
Done with week 28
Done with week 27
Done with week 26
Done with week 25
Done with week 24
Done with week 23
Done with week 22
Done with week 21
Done with week 20
Done with week 19
Done with week 18
Done with week 17
Done with week 16
Done with week 15
Done with week 14
Done with week 13
Done with week 12
Done with week 11
Done with week 10
Done with week 9
Done with week 8
Done with week 7
Done with week 6
Done with week 5
Done with week 4
Done with week 3
Done with week 2
Done with week 1


In [33]:
mta.shape

(10258959, 11)

In [34]:
mta.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10258959 entries, 0 to 10258958
Data columns (total 11 columns):
C/A                                                                     object
UNIT                                                                    object
SCP                                                                     object
STATION                                                                 object
LINENAME                                                                object
DIVISION                                                                object
DATE                                                                    object
TIME                                                                    object
DESC                                                                    object
ENTRIES                                                                 int64
EXITS                                                                   int64
dtypes: int64(2), 

We need to change DATE attribute's datatype to date time

In [42]:
mta["DATE"] = pd.to_datetime(mta["DATE"])

In [45]:
mta.dtypes

C/A                 object
UNIT                object
SCP                 object
STATION             object
LINENAME            object
DIVISION            object
DATE        datetime64[ns]
TIME                object
DESC                object
ENTRIES              int64
EXITS                int64
dtype: object

In [37]:
mta.columns

Index(['C/A', 'UNIT', 'SCP', 'STATION', 'LINENAME', 'DIVISION', 'DATE', 'TIME',
       'DESC', 'ENTRIES',
       'EXITS                                                               '],
      dtype='object')

### So there are trailing spaces in EXITS column... We'll rename it.

In [38]:
mta.rename(columns={'EXITS                                                               ':'EXITS'}, inplace=True)

In [46]:
mta.columns

Index(['C/A', 'UNIT', 'SCP', 'STATION', 'LINENAME', 'DIVISION', 'DATE', 'TIME',
       'DESC', 'ENTRIES', 'EXITS'],
      dtype='object')

In [47]:
len(mta["DATE"].unique())

364

#### The data is ready, however, it is best to save it so in case we need to shutdown the kernel or if it crashes, we won't need to re-run all the above script.
#### Pickling is the technique that is used to store python objects like lists, dictionaries, class objects, etc.
#### Pickling is the serializing and de-serializing of python objects to a byte stream. Unpicking is the opposite.

In [4]:
import pickle

In [52]:
pickle_out = open("mta_dataset","wb")

In [53]:
pickle.dump(mta, pickle_out)

In [5]:
#Open the pickle file and load it to the variable
pickle_in = open("mta_dataset","rb")
mta = pickle.load(pickle_in)


### So now we have the data for year 2017, we are ready for the analysis part.

***

# Part II - Analysis
## Understanding the data
#### Some details about the attributes

* C/A      = Control Area (A002) <br />

* UNIT     = Remote Unit for a station (R051)

* SCP      = Subunit Channel Position represents an specific address for a device (02-00-00)

* STATION  = Represents the station name the device is located at

* LINENAME = Represents all train lines that can be boarded at this station. 
    * Normally lines are represented by one character.  LINENAME 456NQR repersents train server for 4, 5, 6, N,Q, and R trains.

* DIVISION = Represents the Line originally the station belonged to BMT, IRT, or IND   

* DATE     = Represents the date (YYYY-MM-DD)

* TIME     = Represents the time (hh:mm:ss) for a scheduled audit event

* DESC     = Represent the "REGULAR" scheduled audit event (Normally occurs every 4 hours)
               * 1. Audits may occur more that 4 hours due to planning, or troubleshooting activities. 
               * 2. Additionally, there may be a "RECOVR AUD" entry: This refers to a missed audit that was recovered. 

* ENTRIES  = The comulative entry register value for a device

* EXITS    = The cumulative exit register value for a device

**_The entries and exits aren’t counts per interval, but equivalent to an “odometer” reading for each device._**


** Which column represents a unique turnstile? **

At first I thought that the UNIT attribute is the turnstile, however, I found out that there are far fewer units at every station on an average just 2 or 3. Even a small station generally has at least 4 - 6 turnstiles and bigger ones have multiple entry/exit points so there could be 20 - 30 turnstiles.

After some Googling and going through some forums, I found that the SCP column represents the turnstiles. 


In [25]:
#number of unique units at every station
mta.groupby("STATION").UNIT.nunique()[:10]

STATION
1 AV               1
103 ST             3
103 ST-CORONA      1
104 ST             2
110 ST             1
111 ST             3
116 ST             3
116 ST-COLUMBIA    1
121 ST             1
125 ST             4
Name: UNIT, dtype: int64

In [26]:
mta.groupby("STATION").SCP.nunique().sort_values(ascending=False)[:10]

STATION
PATH NEW WTC       90
FULTON ST          80
34 ST-PENN STA     58
CHAMBERS ST        49
ATL AV-BARCLAY     48
TIMES SQ-42 ST     43
WALL ST            43
GRD CNTRL-42 ST    41
34 ST-HERALD SQ    41
59 ST COLUMBUS     41
Name: SCP, dtype: int64

In [28]:
mta.groupby("STATION").SCP.value_counts()[:50]

STATION        SCP     
1 AV           00-03-00    2204
               00-03-02    2204
               00-03-01    2203
               00-00-00    2202
               00-00-01    2202
               01-00-02    2196
               01-00-03    2196
               01-00-04    2195
               01-00-00    2194
               01-00-01    2193
103 ST         00-00-01    6596
               00-00-02    6596
               00-00-00    6579
               00-03-02    4409
               00-03-01    4404
               00-03-00    4394
103 ST-CORONA  00-00-00    2195
               00-05-00    2195
               00-05-01    2195
               00-06-00    2195
               00-06-01    2195
               00-00-01    2193
               00-00-02    2193
               00-00-03    2193
               00-00-04    2192
104 ST         00-00-01    4344
               00-00-02    4343
               00-00-00    4316
               00-06-01    2183
               00-06-00    2179
               0

**So clearly there are no unique SCP values for every station**

So we need to think of a combination to identify a unique turnstile.

In [29]:
#let's try with station as well as unit
mta.groupby(["STATION","UNIT"]).SCP.value_counts()[:50]


STATION        UNIT  SCP     
1 AV           R248  00-03-00    2204
                     00-03-02    2204
                     00-03-01    2203
                     00-00-00    2202
                     00-00-01    2202
                     01-00-02    2196
                     01-00-03    2196
                     01-00-04    2195
                     01-00-00    2194
                     01-00-01    2193
103 ST         R180  00-03-02    2215
                     00-00-01    2214
                     00-00-00    2213
                     00-00-02    2213
                     00-03-01    2209
                     00-03-00    2200
               R191  00-00-02    2195
                     00-03-01    2195
                     00-00-01    2194
                     00-03-00    2194
                     00-03-02    2194
                     00-00-00    2178
               R314  00-00-00    2188
                     00-00-01    2188
                     00-00-02    2188
103 ST-CORONA  R208 

We can see that SCPs are reapeted among different units.
**So STATION,UNIT,SCP should together identify a unique turnstile.**

**Also, let's check if there is any relation between STATION and C/A**

In [41]:
mta.groupby("STATION").["C/A"].nunique().value_counts()

SyntaxError: invalid syntax (<ipython-input-41-b6d691a02130>, line 1)

In [44]:
mta.rename(columns={'C/A':'CA'}, inplace=True)

In [45]:
mta.head()

,CA,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS
0,A002,R051,02-00-00,59 ST,NQR456W,BMT,2017-12-23,03:00:00,REGULAR,6455840,2184987
1,A002,R051,02-00-00,59 ST,NQR456W,BMT,2017-12-23,07:00:00,REGULAR,6455856,2184995
2,A002,R051,02-00-00,59 ST,NQR456W,BMT,2017-12-23,11:00:00,REGULAR,6455899,2185082
3,A002,R051,02-00-00,59 ST,NQR456W,BMT,2017-12-23,15:00:00,REGULAR,6456038,2185156
4,A002,R051,02-00-00,59 ST,NQR456W,BMT,2017-12-23,19:00:00,REGULAR,6456327,2185197


In [72]:
mta.groupby("STATION").CA.nunique()

STATION
1 AV               2
103 ST             3
103 ST-CORONA      1
104 ST             2
110 ST             2
111 ST             4
116 ST             6
116 ST-COLUMBIA    1
121 ST             1
125 ST             6
135 ST             5
137 ST CITY COL    2
138/GRAND CONC     1
14 ST              7
14 ST-UNION SQ     5
145 ST             5
149/GRAND CONC     2
14TH STREET        1
15 ST-PROSPECT     2
155 ST             2
157 ST             2
161/YANKEE STAD    5
163 ST-AMSTERDM    1
167 ST             3
168 ST             3
169 ST             2
170 ST             3
174 ST             2
174-175 STS        2
175 ST             2
                  ..
SUTPHIN-ARCHER     1
SUTTER AV          1
SUTTER AV-RUTLD    1
THIRTY ST          1
THIRTY THIRD ST    1
TIMES SQ-42 ST     6
TOMPKINSVILLE      1
TREMONT AV         1
TWENTY THIRD ST    1
UNION ST           2
UTICA AV           2
V.CORTLANDT PK     1
VAN SICLEN AV      2
VAN SICLEN AVE     1
VERNON-JACKSON     3
W 4 ST-WASH SQ     2
W 8 S

So every station has at least 1 or more control areas.

We need to check that the control area and station has a one-to-one relationship.

In [79]:
mta.groupby("CA").STATION.nunique().sort_values(ascending=False).head(20)

CA
N408A    2
TRAM2    1
N103     1
N117     1
N116     1
N114     1
N113     1
N112A    1
N111     1
N110     1
N108     1
N102     1
N119     1
N101     1
N100     1
N098     1
N095A    1
N095     1
N094     1
N092     1
Name: STATION, dtype: int64

We can see that all but one CA correspond to one unique station.

So except for control area N408A, there is a one-to-one relationship between control area and station.

In [77]:
len(mta["CA"].unique())

737

In [76]:
mta.groupby("CA").STATION.nunique().value_counts()

1    736
2      1
Name: STATION, dtype: int64

This also proves that there is only one instance of 2 stations having the same control area.

<font color="red">**_So we now we can conclude that the a turnstile can be identified by the combination of (STATION, UNIT, SCP)_**</font>

Also, let's make sure that we remove this exception of CA N408A.

In [82]:
mta.drop(mta[mta["CA"] == "N408A"].index, inplace=True)

In [83]:
mta[mta["CA"] == "N408A"]

,CA,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS


We have successfully removed records with CA = N408A

***

## The next question is getting the actual number of entries and exits
I found out from a forum that the turnstile readings reset every now and then, however, the reading **do not completely reset to 0**.

I found this by plotting the numbers by date and time attributes in Tableau.

One startegy could be to get the difference in consecutive readings taken from a turnstile.

However, we first need to have the dataset sorted in the order of station.

Changing order of the columns...

In [85]:
mta = mta[["STATION","CA","UNIT","SCP", "LINENAME","DIVISION","DATE","TIME","DESC","ENTRIES","EXITS"]]

In [86]:
mta.head()

,STATION,CA,UNIT,SCP,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS
0,59 ST,A002,R051,02-00-00,NQR456W,BMT,2017-12-23,03:00:00,REGULAR,6455840,2184987
1,59 ST,A002,R051,02-00-00,NQR456W,BMT,2017-12-23,07:00:00,REGULAR,6455856,2184995
2,59 ST,A002,R051,02-00-00,NQR456W,BMT,2017-12-23,11:00:00,REGULAR,6455899,2185082
3,59 ST,A002,R051,02-00-00,NQR456W,BMT,2017-12-23,15:00:00,REGULAR,6456038,2185156
4,59 ST,A002,R051,02-00-00,NQR456W,BMT,2017-12-23,19:00:00,REGULAR,6456327,2185197


Sorting values by STATION, CA, UNIT, SCP, DATE, TIME so that we have the entry/exit numbers in correct order before we get the difference between consecutive rows...

In [87]:
mta = mta.sort_values(["STATION", "CA", "UNIT", "SCP", "DATE", "TIME"])

In [88]:
mta.head()

,STATION,CA,UNIT,SCP,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS
10093563,1 AV,H007,R248,00-00-00,L,BMT,2016-12-31,03:00:00,REGULAR,11892866,13050144
10093564,1 AV,H007,R248,00-00-00,L,BMT,2016-12-31,07:00:00,REGULAR,11892926,13050308
10093565,1 AV,H007,R248,00-00-00,L,BMT,2016-12-31,11:00:00,REGULAR,11893253,13050807
10093566,1 AV,H007,R248,00-00-00,L,BMT,2016-12-31,15:00:00,REGULAR,11893870,13051562
10093567,1 AV,H007,R248,00-00-00,L,BMT,2016-12-31,19:00:00,REGULAR,11894473,13052475


In [89]:
mta["realEntries"] = mta["EXITS"].diff()

In [92]:
#oops we need to change the name from realEntries to actualExits
mta.rename(columns={"realEntries":"actualExits"}, inplace=True)
mta.head()

,STATION,CA,UNIT,SCP,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS,actualExits
10093563,1 AV,H007,R248,00-00-00,L,BMT,2016-12-31,03:00:00,REGULAR,11892866,13050144,NaN
10093564,1 AV,H007,R248,00-00-00,L,BMT,2016-12-31,07:00:00,REGULAR,11892926,13050308,164.0
10093565,1 AV,H007,R248,00-00-00,L,BMT,2016-12-31,11:00:00,REGULAR,11893253,13050807,499.0
10093566,1 AV,H007,R248,00-00-00,L,BMT,2016-12-31,15:00:00,REGULAR,11893870,13051562,755.0
10093567,1 AV,H007,R248,00-00-00,L,BMT,2016-12-31,19:00:00,REGULAR,11894473,13052475,913.0


In [93]:
mta["actualEntries"] = mta["ENTRIES"].diff()

In [94]:
mta.head()

,STATION,CA,UNIT,SCP,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS,actualExits,actualEntries
10093563,1 AV,H007,R248,00-00-00,L,BMT,2016-12-31,03:00:00,REGULAR,11892866,13050144,NaN,NaN
10093564,1 AV,H007,R248,00-00-00,L,BMT,2016-12-31,07:00:00,REGULAR,11892926,13050308,164.0,60.0
10093565,1 AV,H007,R248,00-00-00,L,BMT,2016-12-31,11:00:00,REGULAR,11893253,13050807,499.0,327.0
10093566,1 AV,H007,R248,00-00-00,L,BMT,2016-12-31,15:00:00,REGULAR,11893870,13051562,755.0,617.0
10093567,1 AV,H007,R248,00-00-00,L,BMT,2016-12-31,19:00:00,REGULAR,11894473,13052475,913.0,603.0


In [95]:
#fill NaN with 0
mta.fillna(0, inplace=True)

In [98]:
mta.head()

,STATION,CA,UNIT,SCP,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS,actualExits,actualEntries
10093563,1 AV,H007,R248,00-00-00,L,BMT,2016-12-31,03:00:00,REGULAR,11892866,13050144,0.0,0.0
10093564,1 AV,H007,R248,00-00-00,L,BMT,2016-12-31,07:00:00,REGULAR,11892926,13050308,164.0,60.0
10093565,1 AV,H007,R248,00-00-00,L,BMT,2016-12-31,11:00:00,REGULAR,11893253,13050807,499.0,327.0
10093566,1 AV,H007,R248,00-00-00,L,BMT,2016-12-31,15:00:00,REGULAR,11893870,13051562,755.0,617.0
10093567,1 AV,H007,R248,00-00-00,L,BMT,2016-12-31,19:00:00,REGULAR,11894473,13052475,913.0,603.0


In [97]:
mta.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10239238 entries, 10093563 to 173643
Data columns (total 13 columns):
STATION          object
CA               object
UNIT             object
SCP              object
LINENAME         object
DIVISION         object
DATE             datetime64[ns]
TIME             object
DESC             object
ENTRIES          int64
EXITS            int64
actualExits      float64
actualEntries    float64
dtypes: datetime64[ns](1), float64(2), int64(2), object(8)
memory usage: 1.4+ GB


Ohhh the two new columns are now float. So let's get them into int.

In [103]:
mta["actualEntries"] = mta["actualEntries"].astype(int)

In [105]:
mta["actualExits"] = mta["actualExits"].astype(int)

In [106]:
mta.head()

,STATION,CA,UNIT,SCP,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS,actualExits,actualEntries
10093563,1 AV,H007,R248,00-00-00,L,BMT,2016-12-31,03:00:00,REGULAR,11892866,13050144,0,0
10093564,1 AV,H007,R248,00-00-00,L,BMT,2016-12-31,07:00:00,REGULAR,11892926,13050308,164,60
10093565,1 AV,H007,R248,00-00-00,L,BMT,2016-12-31,11:00:00,REGULAR,11893253,13050807,499,327
10093566,1 AV,H007,R248,00-00-00,L,BMT,2016-12-31,15:00:00,REGULAR,11893870,13051562,755,617
10093567,1 AV,H007,R248,00-00-00,L,BMT,2016-12-31,19:00:00,REGULAR,11894473,13052475,913,603


There is one more thing...
Since the turnstile entry/exit numbers never reset to 0, we would have some negative values in the actualEntries and actualExits column.

In [108]:
(mta["actualEntries"] < 0).value_counts()

False    10155124
True        84114
Name: actualEntries, dtype: int64

In [109]:
(mta["actualExits"] < 0).value_counts()

False    10170856
True        68382
Name: actualExits, dtype: int64

We will have to convert these negative values to 0 so as to get the maximum possible correct numbers of actual entries and exits. Unfortunately, there is no other way to find the real numbers when the counter on the turnstile resets.

In [132]:
mta.loc[(mta[mta["actualEntries"] < 0]).index, "actualEntries"] = 0

In [134]:
mta.loc[(mta[mta["actualExits"] < 0]).index, "actualExits"] = 0

## Finally the dataset is ready :)
### Let's answer a few questions now

## Which station has the most number of units?

In [136]:
mta.groupby("STATION").UNIT.nunique().sort_values(ascending=False)[:10]

STATION
23 ST              6
86 ST              5
CANAL ST           5
GRD CNTRL-42 ST    4
125 ST             4
34 ST-PENN STA     4
FULTON ST          3
96 ST              3
PROSPECT AV        3
18 AV              3
Name: UNIT, dtype: int64

## What is the total number of entries & exits across the subway system for August 1, 2017?

In [137]:
mta[mta["DATE"] == "2017-08-01"].actualEntries.sum()

72729379

In [138]:
mta[mta["DATE"] == "2017-08-01"].actualExits.sum()

4364669

## What station was the busiest on July 4, 2017? What turnstile was the busiest on that date?

In [139]:
mta["Busyness"] = mta["actualEntries"] + mta["actualExits"]

In [149]:
mta[mta["DATE"]=="2017-07-04"].head()

,STATION,CA,UNIT,SCP,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS,actualExits,actualEntries,Busyness
4983598,1 AV,H007,R248,00-00-00,L,BMT,2017-07-04,00:00:00,REGULAR,12486632,13796828,582,285,867
4983599,1 AV,H007,R248,00-00-00,L,BMT,2017-07-04,04:00:00,REGULAR,12486686,13796995,167,54,221
4983600,1 AV,H007,R248,00-00-00,L,BMT,2017-07-04,08:00:00,REGULAR,12486740,13797218,223,54,277
4983601,1 AV,H007,R248,00-00-00,L,BMT,2017-07-04,12:00:00,REGULAR,12487082,13797599,381,342,723
4983602,1 AV,H007,R248,00-00-00,L,BMT,2017-07-04,16:00:00,REGULAR,12487574,13798270,671,492,1163


In [151]:
mta[mta["DATE"] == "2017-07-04"].groupby("STATION").Busyness.sum().sort_values(ascending=False)[:10]

STATION
DITMAS AV          16320926
34 ST-PENN STA       127368
34 ST-HERALD SQ      112015
GRD CNTRL-42 ST       99586
TIMES SQ-42 ST        95084
14 ST-UNION SQ        90588
42 ST-PORT AUTH       88024
86 ST                 75420
125 ST                74078
FULTON ST             72663
Name: Busyness, dtype: int64

So Ditmas Av station was the busiest one on Independance Day in 2017.

In [157]:
mta[(mta["DATE"] == "2017-07-04") & (mta["STATION"] == "DITMAS AV")].groupby(["UNIT","SCP"]).Busyness.sum().sort_values(ascending=False)

UNIT  SCP     
R420  01-06-01    16317812
      00-00-02        1009
      00-00-00         986
      00-00-01         578
      01-06-02         309
      01-06-00         230
      01-04-00           2
      01-04-01           0
Name: Busyness, dtype: int64

**Turnstile _01-06-01_ was the busiest at Ditmas Av station of 4th of July.**